# Optimizer

In [4]:
import numpy as np
import scipy as sp
from scipy.optimize import minimize
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
from matplotlib.lines import Line2D
from scipy.special import lambertw
import warnings
import pickle
from types import SimpleNamespace
import ipynb.fs.defs.Functions_Equations_Equilibrium_Simulation as baseFuncs

### Optimizer Function

In [3]:
def nudgeIC(x0, nudge = 0.01, bnds = ((0,1), (0,1)), eps = 0.0001):
    """
    This function just slightly nudges the initial condition in case it is right against the boundary.
    NOTE: x0 is the initial position for the pair (d,c).
    """
    boundaryNudge = 0
    if np.abs(x0[0]+x0[1] - 1) < eps:
        if x0[0] < 0.5:
            x0[0] = x0[0] + nudge
            x0[1] = x0[1] - 2*nudge #Times two to decrease the value of the sum
        else:
            x0[0] = x0[0] - 2*nudge
            x0[1] = x0[1] + nudge            
        boundaryNudge = 1
    return x0, boundaryNudge

In [4]:
def optimize_simulate(dc_pair, para, tau = 100, T = 100, optimize = True):
    para['d'] = dc_pair[0]
    para['c'] = dc_pair[1]
    return baseFuncs.simulate(para, True, tau, T, optimize = optimize)

In [1]:
def optimizer(para, tau, T, t_max, start_from_eql = False, loadPrevious = False, saveData = True, 
              loadName = "", saveName = "", saveLog = False, run_to_eql = False, tol = 1e-7):
    """
    optimizer()   The optimization function used for finite time-horizons
                    For a single-step optimization:    tau == T == sim_t_max
                    For dynamic optimization:          tau < sim_t_max    and    tau <= T
    T             Time-horizon of firm
    tau           Decision period of firm
    sim_t_max     Length of time for which the firm is dynamically optimizing
    para          Parameter dictionary
    loadPrevious  Boolean of whether to load a previous simulation's data from a Pickle
    saveData      Boolean of whether to save the current simulation's data to a Pickle
    loadName      Name of pickle to read if loadPrevious
    saveName      Destination file name if saveData
    plotContour   Creates contour plot of the cumulative utility for the period T (given the current state 
                  of the system) and plots the optimal strategy position for each step taken
    """
    
    
    if tau > T:
        raise ValueError("tau must be less than or equal to T")
    if tau > t_max:
        raise ValueError("tau must be less than or equal to t_max")
    if T%tau != 0:
        raise ValueError("T must be a multiple of tau")
    if loadPrevious and loadName == "":
        raise ValueError("Please provide file name to load from")
    if saveData and saveName == "":
        raise ValueError("Please provide file name to save to")

    # Find optimal equilibrium position
    bnds = ((0, 1), (0, 1))
    con = lambda x: x[0]+x[1] 
    cons = {'type':'ineq', 'fun': con}
    res = minimize(baseFuncs.equilGeneralAnalytic, [0.01, 0.01], args = (para, -1), bounds = bnds, constraints = cons)
    
    # Initialize/load data
    time_tracker = 0
    nudge = 0.01
    counter = 1
    
    if loadPrevious:
        file = open(loadName, 'rb')
        data = pickle.load(file)
        file.close()
        para['init_R_u'] = data['R_u_vals'][-1]
        para['init_R_o'] = data['R_o_vals'][-1]
        data['t_max'] += t_max
    else:
        data = {"opt_d" : [], "opt_c" : [], "opt_gamma_c" : [], "opt_gamma_p" : [], "utility_vals" : [], 
                "R_u_vals" : [], "R_o_vals" : [], "R_u_init" : -1, "R_o_init" : -1, 
                "prop_Ro" : [], "prop_wA" : [],
               "w_D_vals" : [], "x0" : [0.25, 0.25], "T" : T, "tau" : tau, "t_max" : t_max, "time_tracker" : 0, "start_from_eql" : start_from_eql, 
                "alpha" : para['alpha'], "beta" : para['beta'], "r_d" : para['r_d'], "discount_rate" : para['discount_rate'], "dt" : para['dt'],
                "max_rules" : int(para['w_max']/para['k_A']), "std_potential_ds" : [], "std_potential_cs" : []}
        if start_from_eql:
            para["init_R_u"], para["init_R_o"] = baseFuncs.equilGeneralAnalytic((res.x[0], res.x[1]), para, invalidNan = True, return_rules = True)
        else:
            para["init_R_u"] = 1
            para["init_R_o"] = 0
        
    if res.success and res.fun != 0:
        data['equil_d'] = res.x[0]
        data['equil_c'] = res.x[1] 
        if not loadPrevious:
            data['x0'] = res.x
    else:
        data['equil_d'] = -1
        data['equil_c'] = -1

    if start_from_eql:
        equil_rule_vals = baseFuncs.equilGeneralAnalytic(res.x, para, return_rules = True)
        para['init_R_u'] = equil_rule_vals[0]
        para['init_R_o'] = equil_rule_vals[1]

    
    # Open data file for optimization log
    if saveLog:
        if loadPrevious:
            logFile = open(f"LogFile for T={T}, tau = {tau}, discount rate = {para['discount_rate']}, tmax = {data['t_max']}.txt","a")
        else:
            logFile = open(f"LogFile for T={T}, tau = {tau}, discount rate = {para['discount_rate']}, tmax = {data['t_max']}.txt","w")
        logFile.write("timestep, d, c, decision period utility, R_u, R_o, Boundary IC (nudge), "
                      +"successCount, var l, var c\n") 
            
    reses = [] # I tried to pluralize "res" and this was better than "ressies"

    # While loop for optimizer
    old_Ru = -1
    old_Ro = -1
    Ru = para['init_R_u']
    Ro = para['init_R_o']
    continue_condition = True
    while continue_condition:
        if run_to_eql:
            if abs(Ru-old_Ru) > tol or abs(Ro-old_Ro) > tol:
                continue_condition = True
            else:
                continue_condition = False
                if loadPrevious:
                    data['t_max'] += time_tracker
                else:
                    data['t_max'] = time_tracker
                break
            if t_max-time_tracker <= 0:
                continue_condition = False
                break
        else:
            if t_max-time_tracker > 0:
                continue_condition = True
            else:
                continue_condition = False
                break
        
        # Checking if the initial condition was invalid or on boundary    
        boundaryNudge = 0
        
        # Checking if IC is on boundary
        data['x0'], boundaryNudge = nudgeIC(data['x0'], nudge, bnds)
  
        # Find optimal next step
        max_U = -1
        success_count = 0
        x0s = [data['x0'], [data['x0'][0]/2, data['x0'][1]/2], [(2-data['x0'][0])/2, (2-data['x0'][1])/2],
               [data['x0'][0]/2, (2-data['x0'][1])/2], [(2-data['x0'][0])/2, data['x0'][1]/2], 
               [data['x0'][0]/2, data['x0'][1]], [(2-data['x0'][0])/2, data['x0'][1]],
               [data['x0'][0], (2-data['x0'][1])/2], [data['x0'][0], data['x0'][1]/2]]
        potential_ds = []
        potential_cs = []
        max_d = -1
        max_c = -1
        
        for x0 in x0s:
            warnings.filterwarnings('error')
            try: # The minimize function evaluates over all of the 
                res = minimize(optimize_simulate, x0, args = (para, tau, T, True), 
                           bounds = bnds, method = "Nelder-Mead") # constraints = cons) # 
            except Warning:
                print(x0)
                #res = SimpleNamespace(**{'success': False, 'fun': 0})
            
            if res.success and res.fun != 0:
                if res.x[0] < 0 or res.x[1] < 0:
                    print(res)
                potential_ds.append(res.x[0])
                potential_cs.append(res.x[1])
                if -1*res.fun > max_U:
                    max_U = -1*res.fun
                    max_d = res.x[0]
                    max_c = res.x[1]
                    max_res = res
                success_count += 1       
            
        # Something here
        R_us, R_os, w_As, w_Ds, Us, _, timesteps = optimize_simulate((max_d, max_c), para, tau, tau, False)
        reses.append(max_res)

        # Updating data
        data['opt_d'] += list(np.ones(len(R_us))*max_d)
        data['opt_c'] += list(np.ones(len(R_us))*max_c)
        data['x0'] = [max_d, max_c]
        data['std_potential_ds'].append(np.std(potential_ds))
        data['std_potential_cs'].append(np.std(potential_cs))
        data['utility_vals'] += Us
        data['w_D_vals'] += w_Ds
        data['R_u_vals'] += R_us
        data['R_o_vals'] += R_os
        para['init_R_u'] = R_us[-1]
        para['init_R_o'] = R_os[-1]
        R_tot_vals = np.asarray(R_us)+np.asarray(R_os)
        prop_Ro_vals = np.asarray(R_os)/(R_tot_vals)
        prop_wA_vals = para['k_A']*R_tot_vals/(np.ones(np.asarray(R_tot_vals).shape)*para['w_max'])
        data['prop_Ro'] += list(prop_Ro_vals)
        data['prop_wA'] += list(prop_wA_vals)
        old_Ru = Ru
        old_Ro = Ro
        Ru = R_us[-1]
        Ro = R_os[-1]
        
        
        # If not successful       
        if success_count == 0:
            file = open(f"ErrorSave", 'wb')
            pickle.dump(data, file)
            file.close()
            logFile.write(f"Process ended early due to {res.message}")
            print(f"Ended on step: {time_tracker/tau}. {para['init_R_u']} {para['init_R_o']} {res.x} Error message: {res}")
            return data
        
        # Checking if the firm has experienced bureaucratic death
        w_A_val = para['k_A']*(data['R_u_vals'][-1] + data['R_u_vals'][-1])
        if w_A_val > 0.999*para['w_max']:
            print(f"Bureaucratic death. w_A = {w_A_val} > 99% of w_max.")
            logFile.write("Simulation terminated. The firm has experienced bureaucratic death.")
            break
            
        # Update time    
        time_tracker += tau

        # Updating log
        if saveLog:
            logFile.write(f"{int(time_tracker/tau)}, {max_d}, {max_c}, {max_U}, {para['init_R_u']}, " 
                          f"{para['init_R_o']}, {nudge*boundaryNudge}, {success_count}, "
                          f"{np.std(potential_ds)}, {np.std(potential_cs)}\n")
    
    # Close log
    if saveLog:
        logFile.close()
           
    # Reset init_R_u and init_R_o
    para['init_R_u'] = 1
    para['init_R_o'] = 0
    data['time_tracker'] = time_tracker
    
    # Save data    
    if saveData:
        file = open(saveName, 'wb')
        pickle.dump(data, file)
        file.close()
        
    return data